In [ ]:
from math import sqrt

import pandas as pd
from bokeh.plotting import output_notebook, figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import Set1_3

output_notebook()

In [ ]:
# https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT
demo_df = pd.read_sas('DEMO_G.XPT')[['SEQN', 'RIAGENDR', 'RIDAGEYR']].rename(columns={
        'RIAGENDR':'gender',
        'RIDAGEYR':'age',
    }).set_index('SEQN')
demo_df.gender = demo_df.gender.astype(int)
demo_df.age = demo_df.age.astype(int)
demo_df.head()

In [ ]:
# https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/MGX_G.XPT
grip_df = pd.read_sas('MGX_G.XPT')[['SEQN', 'MGDCGSZ']].rename(columns={
        'MGDCGSZ':'grip_strength',
    }).set_index('SEQN')
grip_df.grip_strength = grip_df.grip_strength.round()
grip_df.head()

In [ ]:
df = demo_df.join(grip_df).dropna().reset_index()
df.head()

In [ ]:
plot_df = df.groupby(['gender', 'age', 'grip_strength'], as_index=False).count().rename(columns={'SEQN':'count'})
plot_df['color'] = plot_df['gender'].map(lambda g: Set1_3[g])
plot_df['radius'] = plot_df['count'].map(lambda n: sqrt(n) / 2)
plot_df['gender'] = plot_df['gender'].map(lambda g: ['', 'Male', 'Female'][g])
plot_df.head()

In [ ]:
f = figure(
    width=900,
    height=900,
    x_axis_label='Age (years)',
    y_range=[0, 200],
    y_axis_label='Combined Grip Strength (kg)',
)
f.circle(
    x='age',
    y='grip_strength',
    radius='radius',
    color='color',
    line_width=1.25,
    line_alpha=0.6,
    fill_alpha=0,
    legend='gender',
    source=ColumnDataSource(plot_df),
)
f.add_tools(HoverTool(tooltips=[
            ('Gender', '@gender'),
            ('Age', '@age'),
            ('Grip Strength', '@grip_strength'),
            ('Count', '@count'),
        ]))
show(f)

In [ ]:
plot_df = df.groupby(['gender', 'age']).count().rename(columns={'SEQN':'count'})[['count']]
for i in range(0, 101, 25):
    quantile = i / 100
    qdf = df.groupby(['gender', 'age']).quantile(quantile)[['grip_strength']]
    qdf = qdf.rename(columns={'grip_strength':'q{:03d}'.format(round(100 * quantile))})
    plot_df = plot_df.join(qdf)
plot_df = plot_df.reset_index()
plot_df['color'] = plot_df['gender'].map(lambda g: Set1_3[g])
plot_df['gender'] = plot_df['gender'].map(lambda g: ['', 'Male', 'Female'][g])
plot_df.head()

In [ ]:
f = figure(
    width=900,
    height=900,
    x_axis_label='Combined Grip Strength (kg)',
    y_range=[0, 200],
    y_axis_label='Age (years)',
)
f.segment(
    x0='age',
    y0='q100',
    x1='age',
    y1='q075',
    color='color',
    line_alpha=0.75,
    source=ColumnDataSource(plot_df),
)
renderer = f.square(
    x='age',
    y='q050',
    color='color',
    legend='gender',
    source=ColumnDataSource(plot_df),
)
f.segment(
    x0='age',
    y0='q025',
    x1='age',
    y1='q000',
    color='color',
    line_alpha=0.75,
    source=ColumnDataSource(plot_df),
)
f.add_tools(HoverTool(renderers=[renderer], tooltips=[
            ('Gender', '@gender'),
            ('Age', '@age'),
            ('Max', '@q100'),
            ('Q1', '@q075'),
            ('Median', '@q050'),
            ('Q3', '@q025'),
            ('Min', '@q000'),
        ]))
show(f)